In [1]:
print("all okay")

all okay


In [2]:
from langchain_groq import ChatGroq

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
model = ChatGroq(model='qwen/qwen3-32b')

In [5]:
query = "What is the capital of France?"
response = model.invoke(query).content

print(f"Response: {response}")


Response: <think>
Okay, so the question is asking for the capital of France. Let me think. I remember that France is a country in Europe, right? I think the capital is Paris. Wait, is that correct? I've heard Paris mentioned a lot in relation to France. Let me try to recall. There's the Eiffel Tower, the Louvre Museum, and other famous landmarks in Paris. Yeah, those are all in France. I don't think any other cities in France are capitals of regions or anything else... Wait, Lyon is a big city, and Marseille too, but they're not the capital. Oh, and Nice, but definitely not the capital. So it must be Paris. Let me just double-check. I've also heard Paris referred to as the capital of France in news reports and movies. Yeah, I'm pretty confident it's Paris. I don't think there's any confusion with another city here. So the answer should be Paris.
</think>

The capital of France is **Paris**. 

Paris is renowned for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and No

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [7]:
embedding_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001',)

In [8]:
emd = embedding_model.embed_query("What is the capital of France?")
len(emd)

768

### 1. Data Ingestion

In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
import os

In [11]:
os.getcwd()

'c:\\Users\\Kranthi\\OneDrive\\Desktop\\LLMOPS_Krishnaik\\Projects\\Document_Portal\\notebook'

In [12]:
file_path = os.path.join(os.getcwd(), 'data', 'sample.pdf')

In [13]:
loader = PyPDFLoader(file_path=file_path)

In [14]:
documents = loader.load()

In [ ]:
doc